<div>
    <h1 align="center">XGBoost & CatBoost</h1>    
    <h1 align="center">Tabular Playground Series - Aug 2021</h1> 
    <h4 align="center">By: Somayyeh Gholami & Mehran Kazeminia</h4>
</div>

<div class="alert alert-success">  
</div>

<div class="alert alert-success">
    <h1 align="center">If you find this work useful, please don't forget upvoting :)</h1>
</div>

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px

%matplotlib inline

In [ ]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

import optuna

In [ ]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import LabelEncoder
from mlxtend.preprocessing import minmax_scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneGroupOut


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

<div class="alert alert-success">  
</div>

## Data Set

In [ ]:
DF1 = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')

DF2 = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')

SAM = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')

In [ ]:
MV1 = DF1.isnull().sum()
MV2 = DF2.isnull().sum()

print(f'Missing Value 1:  {MV1[MV1 > 0]}')
print(f'Missing Value 2:  {MV2[MV2 > 0]}')

In [ ]:
display(DF1, DF2)

# display(DF1.describe().transpose())
# display(DF2.describe().transpose())

In [ ]:
print('=' * 40)
DF1.info(memory_usage='deep')
print('=' * 40)
DF2.info(memory_usage='deep')
print('=' * 40)

<div class="alert alert-success">  
</div>

In [ ]:
data1 = DF1.copy()
data2 = DF2.copy()

columns = data2.columns[1:]
display(columns)

In [ ]:
data1['loss'].value_counts().plot(figsize=(16, 8), kind='bar')

In [ ]:
data1['loss'].value_counts().plot(figsize=(10, 10), kind='pie')

data1['loss'].value_counts(normalize=True)

<div class="alert alert-success">  
</div>

In [ ]:
X = data1.drop(columns = ['id','loss'])
display(X)

In [ ]:
y = data1.loss
display(y)

In [ ]:
display(y.min() , y.max())

In [ ]:
XX = data2.drop(columns = ['id'])
display(XX)

<div class="alert alert-success">  
</div>

In [ ]:
yc = y.copy()

yc = np.clip(yc, 0, 1)
display(yc)

In [ ]:
display(yc.min() , yc.max())

In [ ]:
yc.value_counts().plot(figsize=(4, 4), kind='bar')

In [ ]:
yc.value_counts().plot(figsize=(5, 5), kind='pie')

yc.value_counts(normalize=True)

<div class="alert alert-success">  
</div>

## Split

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.50, random_state=123) 

val_X.to_csv("val_X.csv",index=False)
val_y.to_csv("val_y.csv",index=False)

In [ ]:
train_X, val_X, train_yc, val_yc = train_test_split(X, yc, test_size=0.50, random_state=123)

val_yc.to_csv("val_yc.csv",index=False)

<div class="alert alert-success">  
</div>

## Scaling

In [ ]:
X_scaled = minmax_scaling(X, columns=X.columns)
# display(X_scaled)

In [ ]:
XX_scaled = minmax_scaling(XX, columns=XX.columns)
# display(XX_scaled)

<div class="alert alert-success">  
</div>

## XGBRegressor

### Validation Model - 1

In [ ]:
model1v = XGBRegressor(max_depth=7,
                       n_estimators=2500,
                       learning_rate=0.008,
                       subsample=0.84,
                       booster= 'gbtree',
                       tree_method= 'gpu_hist',
                       colsample_bytree= 0.70,
                       reg_lambda= 5,
                       reg_alpha= 32,
                       n_jobs= 4,  
                       alpha=0.5,
                       random_state=123)                                  
    
model1v.fit(train_X, train_y)
oof_pred1 = model1v.predict(val_X)
oof_pred1 = np.clip(oof_pred1, y.min(), y.max())

print(30 * '=')
print(f'Mean Error: {np.sqrt(mean_squared_error(val_y, oof_pred1))}')
print(30 * '=')

In [ ]:
model1v.feature_importances_

## Model - 1

In [ ]:
model1 = XGBRegressor(max_depth=7,                     
                      n_estimators=2500,
                      learning_rate=0.008,
                      subsample=0.84,
                      booster= 'gbtree',
                      tree_method= 'gpu_hist',
                      colsample_bytree= 0.70,
                      reg_lambda= 5,
                      reg_alpha= 32,
                      n_jobs= 4,            
                      alpha=0.5,
                      random_state=123)   

model1.fit(X, y)
pred1 = model1.predict(XX)
pred1 = np.clip(pred1, y.min(), y.max())
display(pred1, pred1.shape) 

In [ ]:
sub1 = SAM.copy()

sub1.iloc[:, 1] = pred1.data
display(sub1)

In [ ]:
sub1.to_csv("submission_xgb.csv",index=False)
# Public Score: 7.87965

<div class="alert alert-success">  
</div>

## CatBoostRegressor

### Validation Model - 2

In [ ]:
model2v = CatBoostRegressor(depth=6,
                            iterations=1600,
                            learning_rate=0.024,
                            l2_leaf_reg=20,
                            random_strength=1.5,
                            grow_policy='Depthwise',
                            leaf_estimation_method='Newton', 
                            bootstrap_type='Bernoulli',
                            thread_count=4,
                            verbose=False,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            od_type='Iter',
                            task_type='GPU',
                            early_stopping_rounds=500,
                            random_state=123)    

model2v.fit(train_X, train_y, verbose=200)
oof_pred2 = model2v.predict(val_X)
oof_pred2 = np.clip(oof_pred2, y.min(), y.max())

print(30 * '=')
print(f'Mean Error: {np.sqrt(mean_squared_error(val_y, oof_pred2))}')
print(30 * '=')

In [ ]:
model2v.feature_importances_

## Model - 2

In [ ]:
model2 = CatBoostRegressor(depth=6,                     
                           iterations=1600,
                           learning_rate=0.024,
                           l2_leaf_reg=20,
                           random_strength=1.5,
                           grow_policy='Depthwise',
                           leaf_estimation_method='Newton', 
                           bootstrap_type='Bernoulli',
                           thread_count=4,
                           verbose=False,
                           loss_function='RMSE',
                           eval_metric='RMSE',
                           od_type='Iter',
                           task_type='GPU',
                           early_stopping_rounds=500,
                           random_state=123)    

model2.fit(X, y)
pred2 = model2.predict(XX)
pred2 = np.clip(pred2, y.min(), y.max())
display(pred2, pred2.shape) 

In [ ]:
sub2 = SAM.copy()

sub2.iloc[:, 1] = pred2.data
display(sub2)

In [ ]:
sub2.to_csv("submission_catboost.csv",index=False)
# Public Score: 7.87995

<div class="alert alert-success">  
</div>

## Feature Importances

In [ ]:
a1 = model1v.feature_importances_
a2 = model2v.feature_importances_

axis_x  = X.columns.values
axis_y1 = minmax_scaling(a1, columns=[0])
axis_y2 = minmax_scaling(a2, columns=[0])

plt.style.use('seaborn-whitegrid') 
plt.figure(figsize=(16, 6), facecolor='lightgray')
plt.title(f'\nX G B o o s t  &  C a t B o o s t\n\nF e a t u r e   I m p o r t a n c e s\n', fontsize=14)  

plt.scatter(axis_x, axis_y1, s=20, label='XGBoost') 
plt.scatter(axis_x, axis_y2, s=20, label='CatBoost')

plt.legend(fontsize=12, loc=2)
plt.show()

<div class="alert alert-success">  
</div>

## Best Blend

In [ ]:
def best_blend(coeff):    
    oof_pred = (oof_pred1 * coeff) + (oof_pred2 * (1.0 - coeff))   
    mae = np.sqrt(mean_squared_error(val_y, oof_pred)) 
    return mae

results = {}
for i in range(0, 11):       
    results[0.1 * i] = best_blend(0.1 * i)  
    
plt.plot(list(results.keys()), list(results.values()))
plt.show()   

In [ ]:
pred = (pred1 * 0.50) + (pred2 * (1.0 - 0.50)) 

In [ ]:
sub3 = SAM.copy()

sub3.iloc[:, 1] = pred.data
display(sub3)

In [ ]:
sub3.to_csv("submission3.csv",index=False)
# Public Score: 7.87752

<div class="alert alert-success">  
</div>

## Final Result

Thanks to: @oxzplvifi https://www.kaggle.com/oxzplvifi/tabular-denoising-residual-network

In [ ]:
path0 = '../input/tps8-787043/submission.csv' 

sub787043 = pd.read_csv(path0)

Thanks to: @dmitryuarov https://www.kaggle.com/dmitryuarov/falling-below-7-87-voting-cb-xgb-lgbm

In [ ]:
path1 = '../input/tps8-786780/voting.csv' 

sub786780 = pd.read_csv(path1)

Thanks to: @tensorchoko https://www.kaggle.com/tensorchoko/tabular-aug-2021-lightgbm

In [ ]:
path2 = '../input/tps8-786132/submit.csv' 

sub786132 = pd.read_csv(path2)

Thanks to: @alexryzhkov https://www.kaggle.com/alexryzhkov/lightautoml-classifier-regressor-mix

In [ ]:
path3 = '../input/tps8-785626/LightAutoML_utilized_submission.csv' 

sub785626 = pd.read_csv(path3)

In [ ]:
def ensembling(main, support, coeff): 
    
    suba  = main.copy() 
    subav = suba.values
       
    subb  = support.copy()
    subbv = subb.values    
           
    ense  = main.copy()    
    ensev = ense.values  
 
    for i in range (len(main)):
        
        pera = subav[i, 1]
        perb = subbv[i, 1]
        per = (pera * coeff) + (perb * (1.0 - coeff))   
        ensev[i, 1] = per
        
    ense.iloc[:, 1] = ensev[:, 1]  
    
    ###############################    
    X  = suba.iloc[:, 1]
    Y1 = subb.iloc[:, 1]
    Y2 = ense.iloc[:, 1]
    
    plt.style.use('seaborn-whitegrid') 
    plt.figure(figsize=(9, 9), facecolor='lightgray')
    plt.title(f'\nE N S E M B L I N G\n')   
      
    plt.scatter(X, Y1, s=1.5, label='Support')    
    plt.scatter(X, Y2, s=1.5, label='Generated')
    plt.scatter(X, X , s=0.1, label='Main(X=Y)')
    
    plt.legend(fontsize=12, loc=2)
    #plt.savefig('Ensembling_1.png')
    plt.show()     
    ###############################   
    ense.iloc[:, 1] = ense.iloc[:, 1].astype(float)
    hist_data = [subb.iloc[:, 1], ense.iloc[:, 1], suba.iloc[:, 1]] 
    group_labels = ['Support', 'Ensembling', 'Main']
    
    fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, show_hist=False, show_rug=False)
    fig.show()   
    ###############################       
    
    return ense      


In [ ]:
sub4 = ensembling(sub787043, sub3, 0.60)

sub5 = ensembling(sub786780, sub4, 0.60)

sub6 = ensembling(sub786132, sub5, 0.50)

sub7 = ensembling(sub785626, sub6, 0.70)

In [ ]:
display(sub7)

In [ ]:
sub4.to_csv("submission4.csv",index=False)
sub5.to_csv("submission5.csv",index=False)
sub6.to_csv("submission6.csv",index=False)

sub7.to_csv("submission_final.csv",index=False)
!ls

<div class="alert alert-success">  
</div>

<div class="alert alert-success">  
</div>